# Przetwarzanie Grafiki i Muzyki - laboratorium nr 5

# Zadanie 1 (2 punkty)

Proszę narysować histogramy dla obrazków "LENA_512.jpg" oraz "Statek_640_505.jpg":

  * Narysuj histogram oddzielnie dla R, G, B
  * Narysuj histogram dla szarego zdjęcia

# Zadanie 2 (2 punkty)

Proszę zaimplementować funkcję zmiany kontrastu obrazu.

Wykorzystaj tablicę LUT:

$$
LUT(i) = \left\{ \begin{array}{ll}
0 & \textrm{gdy } a \left(i - \frac{i_{max}}{2} \right)+\frac{i_{max}}{2}<0\\
a \left(i - \frac{i_{max}}{2} \right)+ \frac{i_{max}}{2} & \textrm{gdy } 0 \leq a \left( i - \frac{i_{max}}{2} \right) + \frac{i_{max}}{2} \leq i_{max} \\
i_{max} & \textrm{gdy } a \left( i - \frac{i_{max}}{2} \right)+ \frac{i_{max}}{2}>i_{max} 
\end{array} \right.
$$


Parametr
  * $a<1$ - zmniejszenie kontrastu,
  * $a=1$ - identyczność,
  * $a>1$ - zwiększenie kontrastu.

Uwagi:

  * Proszę zastosować napisaną funkcję do zdjęcia "LENA_512.jpg" i wykonać ją na luminancji (składowa Y z formatu YCbCr) dla $a\in\{0.5, 1, 1.5\}$, a następnie wyświetlić w formacie RGB z uwzględnieniem kanałów Cb i Cr (po zrobieniu konwersji z YCbCr na RGB).
  * Porównaj histogramy przed i po ściemnieniu.

## Rozciągnięcie histogramu

## Zadanie 3 (2 punkty)

Proszę zaimplementować funkcję rozciągającą histogram.

Wykorzystaj tablicę LUT:

$$ 
LUT(i)= \frac{i_{max} }{ {v_{max} - v_{min}} } \cdot (i - v_{min})
$$

Parametr:

  * $v_{max}$ oznacza maksymalną wartość składowej w obrazie,
  * $v_{min}$ oznacza minimalną wartość składowej w obrazie,
  * $i_{max}$ oznacza maksymalną możliwą wartość składowej w obrazie.

Uwagi:

  * Proszę zastosować do zdjęcia LENA_512. Proszę wykonać na Luminancji (składowa Y z formatu YCbCr) a następnie wyświetlić w formacie RGB z uwzględnieniem kanałów Cb i Cr.
  * Porównaj histogramy przed i po wyrównaniu.



## Wyrównywanie histogramu

## Zadanie 4 (2 punkty) 

Proszę zaimplementować funkcję wyrównującą histogram.

  * Najpierw stwórz dystrybuantę:

$$
D(n)= \frac{h_0+h_1+ \ldots +h_n }{sum}
$$

gdzie:

$h_n$ - to ilość punktów na obrazie o n-tym poziomie szarości,

$sum$ - to liczba wszystkich punktów obrazu
  
  *  Wykorzystaj tablicę LUT:
$$ 
LUT(i) = \frac{D(i)-D_0}{1-D_0} \cdot (k-1)
$$
				
Parametr

$D_0$ - to pierwsza niezerowa wartość dystrybuanty obrazu źródłowego,

$k$ - to liczba możliwych wartości jasności obrazu (zwykle 256).

Uwagi:

  * Proszę zastosować do zdjęcia LENA_512. Proszę wykonać na Luminancji (składowa Y z formatu YCbCr), a następnie wyświetlić w formacie RGB z uwzględnieniem kanałów Cb i Cr.
  * PORÓWNAJ HISTOGRAMY PRZED I PO WYRÓWNANIU.

# Zadanie 5 (2 punkty)

Proszę zastosować do zdjęcia "kobieta.jpg":

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/kobieta.jpg)

  * Rozciągnięcie histogramu
  * Wyrównywanie histogramu

Uwagi:

  * Przed wykonaniem powyższych operacji narysuj histogram i usuń piki odpowiadające wartością skrajnym (tak by móc rozciągnąć histogram)
  * Proszę wykonać na zdjęciu w odcieniach szarości i RGB. 
